<a href="https://colab.research.google.com/github/amgothhrithik/Log-Classification/blob/main/classification_on_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import joblib
lr_model=joblib.load("/content/drive/My Drive/Data/Log Classification/lr_classifier.joblib")

In [ ]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model
model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import re
regex_pattern={
      "System reboot initiated by user User\d+": "System Notification" ,
      "User User\d+ logged (out|in).":"User Action"  ,
      "Backup (started|ended) at \d+":"System Notification",
      "Backup completed successfully.":"System Notification",

      "Disk cleanup completed successfully.":"System Notification",
      "System updated to version \d+":"System Notification",
      "File data_\d+.csv uploaded successfully by user User\d+.":"System Notification",
      "Account with ID \d+ created by User\d+.":"User Action",
  }
def Classification_with_regex(log_meassage,regex_pattern=regex_pattern):
  for pattern, label in regex_pattern.items():
    if re.search(pattern,log_meassage,re.IGNORECASE):
      return label
  return None

In [ ]:
logs = [
    "alpha.osapi_compute.wsgi.server - 12.10.11.1 - API returned 404 not found error",
    "GET /v2/3454/servers/detail HTTP/1.1 RCODE   404 len: 1583 time: 0.1878400",
    "System crashed due to drivers errors when restarting the server",
    "Hey bro, chill ya!",
    "Multiple login failures occurred on user 6454 account",
    "Server A790 was restarted unexpectedly during the process of data transfer"
]

import numpy as np
def classify_log(log):
  label=Classification_with_regex(log)
  if label != None :
    return label
  else:
    embeddings=model.encode(log)
    embeddings=np.array(embeddings).reshape(1, -1)
    y_probs = lr_model.predict_proba(embeddings)
    # print(y_probs)
    if np.max(y_probs)>0.5:
      label=lr_model.predict(embeddings)
    return label


for log in logs:
  print(log)
  label=classify_log(log)
  print("Predicted label:",label)

alpha.osapi_compute.wsgi.server - 12.10.11.1 - API returned 404 not found error
Predicted label: ['HTTP Status']
GET /v2/3454/servers/detail HTTP/1.1 RCODE   404 len: 1583 time: 0.1878400
Predicted label: ['HTTP Status']
System crashed due to drivers errors when restarting the server
Predicted label: ['Critical Error']
Hey bro, chill ya!
Predicted label: None
Multiple login failures occurred on user 6454 account
Predicted label: ['Security Alert']
Server A790 was restarted unexpectedly during the process of data transfer
Predicted label: ['Error']


In [ ]:
!pip install dotenv
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
from dotenv import load_dotenv
from groq import Groq
import os
import re
# Load the .env file from Google Drive
load_dotenv('/content/drive/MyDrive/Data/Log Classification/llm_log.env')

# Access the API key stored in the .env file
client=Groq(api_key = os.getenv('GROQ_API_KEY'))
def classification_llm(log_msg):
  prompt = f'''Classify the log message into one of these categories:
  Workflow Error,Deprecation Warning.
  If you can't figure out a category, use "Unclassified".
  Put the category inside <category> </category> tags.
  Log message: {log_msg}'''

  chat=client.chat.completions.create(
      model="llama-3.3-70b-versatile",
      messages=[{"role":"user","content":prompt}]
  )
  content = chat.choices[0].message.content
  content= re.search(r'<category>(.*)<\/category>', content, flags=re.DOTALL)
  category = "Unclassified"
  if content:
    category = content.group(1)


  return category


In [ ]:
logs = [
    ("ModernCRM", "IP 192.168.133.114 blocked due to potential attack"),
    ("BillingSystem", "User User12345 logged in."),
    ("AnalyticsEngine", "File data_6957.csv uploaded successfully by user User265."),
    ("AnalyticsEngine", "Backup completed successfully."),
    ("ModernHR", "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1 RCODE  200 len: 1583 time: 0.1878400"),
    ("ModernHR", "Admin access escalation detected for user 9429"),
    ("LegacyCRM", "Case escalation for ticket ID 7324 failed because the assigned support agent is no longer active."),
    ("LegacyCRM", "Invoice generation process aborted for order ID 8910 due to invalid tax calculation module."),
    ("LegacyCRM", "The 'BulkEmailSender' feature is no longer supported. Use 'EmailCampaignManager' for improved functionality."),
    ("LegacyCRM", " The 'ReportGenerator' module will be retired in version 4.0. Please migrate to the 'AdvancedAnalyticsSuite' by Dec 2025")
]


In [ ]:
import pandas as pd
test=pd.read_csv("/content/drive/My Drive/Data/Log Classification/test.csv")
test.head()

,source,log_message
0,ModernCRM,"""IP 192.168.133.114 blocked due to potential ..."
1,BillingSystem,"""User 12345 logged in."""
2,AnalyticsEngine,"""File data_6957.csv uploaded successfully by ..."
3,AnalyticsEngine,"""Backup completed successfully."""
4,ModernHR,"""GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/ser..."


In [ ]:
import numpy as np
def classify_log(source,log):
  if source=="LegacyCRM":
    label=classification_llm(log)
  else:
    label=Classification_with_regex(log)
    if label != None :
      return label
    else:
      embeddings=model.encode(log)
      embeddings=np.array(embeddings).reshape(1, -1)
      y_probs = lr_model.predict_proba(embeddings)
      if np.max(y_probs)>0.5:
        label=lr_model.predict(embeddings)[0]
  return label

test_labels=[]
for i in range(len(test)):
  label=classify_log(test["source"][i],test["log_message"][i])
  test_labels.append(label)
test["target_label"]=test_labels

In [ ]:
test

,source,log_message,target_label
0,ModernCRM,"""IP 192.168.133.114 blocked due to potential ...",Security Alert
1,BillingSystem,"""User 12345 logged in.""",Security Alert
2,AnalyticsEngine,"""File data_6957.csv uploaded successfully by ...",System Notification
3,AnalyticsEngine,"""Backup completed successfully.""",System Notification
4,ModernHR,"""GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/ser...",HTTP Status
5,ModernHR,"""Admin access escalation detected for user 9429""",Security Alert
6,LegacyCRM,"""Case escalation for ticket ID 7324 failed be...",Workflow Error
7,LegacyCRM,"""Invoice generation process aborted for order...",Workflow Error
8,LegacyCRM,"""The 'BulkEmailSender' feature is no longer s...",Deprecation Warning
9,LegacyCRM,"""The 'ReportGenerator' module will be retired...",Deprecation Warning
